# Azure AI Agents з підтримкою протоколу контексту моделі (MCP)

Цей блокнот демонструє, як використовувати Azure AI Agents з інструментами протоколу контексту моделі (MCP) для створення інтелектуального агента, який може використовувати зовнішні MCP-сервери для розширених можливостей.


## Встановлення необхідних пакетів NuGet

Спочатку потрібно встановити пакет Azure AI Agents Persistent, який забезпечує основну функціональність для роботи з Azure AI Agents.


## Переваги аутентифікації без ключів

Цей блокнот демонструє **аутентифікацію без ключів**, яка має кілька переваг:
- ✅ **Немає необхідності керувати API-ключами** - Використовує аутентифікацію на основі ідентичності Azure
- ✅ **Підвищена безпека** - Секрети не зберігаються в коді або конфігурації
- ✅ **Автоматична ротація облікових даних** - Azure керує життєвим циклом облікових даних
- ✅ **Доступ на основі ролей** - Використовує Azure RBAC для детального налаштування дозволів

`DefaultAzureCredential` автоматично використовує найкраще доступне джерело облікових даних:
1. Керована ідентичність (під час роботи в Azure)
2. Облікові дані Azure CLI (під час розробки)
3. Облікові дані Visual Studio
4. Змінні середовища (якщо налаштовані)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Встановіть пакет Azure Identity для автентифікації з сервісами Azure за допомогою DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Імпорт необхідних просторів імен

Імпортуйте необхідні простори імен для Azure AI Agents та Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Налаштування клієнта Azure AI Agent (автентифікація без ключа)

Налаштуйте змінні конфігурації та створіть PersistentAgentsClient, використовуючи **автентифікацію без ключа**:
- **projectEndpoint**: Кінцева точка проекту Azure AI Foundry
- **modelDeploymentName**: Назва розгорнутої моделі штучного інтелекту (GPT-4.1 nano)
- **mcpServerUrl**: URL сервера MCP (Microsoft Learn API)
- **mcpServerLabel**: Мітка для ідентифікації сервера MCP
- **DefaultAzureCredential**: Використовує керовану ідентичність, Azure CLI або інші джерела облікових даних (без необхідності API-ключів)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Створення визначення інструменту MCP

Створіть визначення інструменту MCP, який підключається до сервера Microsoft Learn MCP. Це дозволить агенту отримувати доступ до контенту та документації Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Створення AI-агента

Створіть постійного AI-агента із заданою моделлю та інструментами MCP. Агент налаштований на:
- Модель GPT-4.1 nano
- Інструкції щодо використання інструментів MCP для допомоги
- Доступ до сервера Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Створення теми та надсилання повідомлення

Створіть тему розмови та надішліть повідомлення користувачу з питанням про різницю між Azure OpenAI та OpenAI. Це перевірить здатність агента використовувати MCP інструменти для надання точної інформації.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Налаштування ресурсів MCP Tool (безключовий доступ)

Налаштуйте ресурси MCP Tool. Для повністю безключового підходу ви можете видалити спеціальні заголовки, якщо сервер MCP підтримує автентифікацію на основі ідентифікації Azure. У прикладі нижче показано, як додати заголовки, якщо це необхідно, але для безключових сценаріїв вони можуть бути не потрібні.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Запуск агента

Створіть і запустіть процес для обробки повідомлення користувача. Агент використовуватиме налаштовані інструменти та ресурси MCP для створення відповіді.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Моніторинг виконання та обробка затверджень інструментів (без ключів)

Слідкуйте за статусом виконання агента та обробляйте необхідні затвердження інструментів. Цей цикл:
1. Очікує завершення виконання або необхідності дій
2. Автоматично затверджує виклики інструментів MCP, коли це потрібно
3. Для автентифікації без ключів заголовки можуть бути не потрібні, якщо сервер MCP підтримує ідентифікацію Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Відображення результатів розмови

Отримайте та відобразіть усі повідомлення в темі, показуючи як запит користувача, так і відповідь агента. Повідомлення відображаються в хронологічному порядку з позначками часу та індикаторами ролей.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
